## Recommendation project POC

Let's load test data with users, items and ratings 

In [1]:
import pandas as pd

import pyodbc 

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=BOSS;'
                      'Database=BooksModel;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

In [2]:
query = "SELECT UserId FROM Ratings;"
df_users = pd.read_sql(query, conn)

df_users

c:\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,UserId
0,5815
1,5815
2,5815
3,5834
4,5834
...,...
1031134,271284
1031135,271284
1031136,271284
1031137,271284


In [3]:
query = "SELECT ItemId FROM Ratings;"
df_books = pd.read_sql(query, conn)

df_books

c:\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,ItemId
0,135297
1,135270
2,135265
3,62754
4,135408
...,...
1031134,205379
1031135,205695
1031136,205029
1031137,203652


In [4]:
query = "SELECT Rating FROM Ratings;"
df_ratings = pd.read_sql(query, conn)

df_ratings

c:\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Rating
0,2.0
1,3.0
2,3.0
3,-5.0
4,-5.0
...,...
1031134,-5.0
1031135,-5.0
1031136,-5.0
1031137,-5.0


In [5]:
conn.commit()
cursor.close()

## Model

We've implemented User2User Collaborative Filtering recommendation model as our POC. This approach uses slightly modified kNN algorithm. Idea is the following. Every user could be represented as vector - in our simple case - vector of ratings. First element is user's rating of the first item, second one is rating of the second etc. 

<table>
    <tr>
        <td><b>User\Item</b></td>
        <td><i>i<sub>1</sub></i></td>
        <td><i>i<sub>2</sub></i></td>
        <td>...</td>
        <td><i>i<sub>n</sub></i></td>
    </tr>
        <tr>
        <td><i>u<sub>1</sub></i></td>
        <td><i>r<sub>11</sub></i></td>
        <td><i>r<sub>12</sub></i></td>
        <td>...</td>
        <td><i>r<sub>1n</sub></i></td>
    </tr>
    <tr>
        <td><i>u<sub>2</sub></i></td>
        <td><i>r<sub>21</sub></i></td>
        <td><i>r<sub>22</sub></i></td>
        <td>...</td>
        <td><i>r<sub>2n</sub></i></td>
    </tr>
    <tr>
        <td>...</td>
        <td>...</td>
        <td>...</td>
        <td>...</td>
        <td>...</td>
    </tr>
    <tr>
        <td><i>u<sub>m</sub></i></td>
        <td><i>r<sub>m1</sub></i></td>
        <td><i>r<sub>m2</sub></i></td>
        <td>...</td>
        <td><i>r<sub>mn</sub></i></td>
    </tr>
    
</table>

Dimension of a user vector = number of items in dataset. 

To get items, recommended to specific user, let's find k users similar to given, calculate sum (or average) of their vectors and treat their coordinates as item ratings. Then select items with highest ratings. 

How we could say that 2 users are similar or not? Let's use cosine similarity: 

$$s(u, v) = \frac{\sum_{i \in I_u \cap I_v} r_{ui}r_{vi}}{\sqrt{\sum_{i \in I_u} r_{ui} ^2}\sqrt{\sum_{i \in I_v} r_{vi}^2}} $$


$I_u$ - set of items, rated by user $u$

$r_{ui}$ - rating given by user $u$ to item $i$.

Positive similarity between 2 users means that they mostly give the same kind of ratings (positive or negative) to same items. Negative similarity means that users give controversal ratings to same items - one loves item and another hates it.

Because number of users is quite high even in our relatively small dataset (~ 280K) finding neighbors by brute forcing will be quite slow. And if remember that we have ~270K items that would be real pain to calculate.

Now the trick. We expect that similar users will have positive cosine similarity value. And as we see from formula above similarity between 2 users is zero if they didn't have common items in their ratings. Therefore, when searching for neighbors we could limit scope only to <b>users that have rated at least one item rated by given user</b>. Because ratings matrix is very sparse - in our dataset user rated around 4 items and item averagely rated by 4 users this will greatly speed-up algorithm. 

After set of k neighbors $N(u)$ is found, calculate average vector of item ratings

$$
\bar{r} = \frac{1}{|N(u)|}\sum_{v \in N(u)} \bar{u}_{i}
$$

where $\bar{u}_{i}$ - vector of <i>i</i>-th neigbor.

From this vector $\bar{r}$ we take items that have positive coordinates, not rated by our user and that would be our recommendations

## Practical part

Here we split data into train and test, dataframes transformed into more efficient indexed structures, items and users replaced by their indexes. Also rating values shifted from [0, 10] to [-5, 5] range (this is simply because 0/10 means absolute hate and 10/10 - absolute love). Plus corrected minor errors like item in ratings is not present in list of items 

In [6]:
from sklearn.model_selection import train_test_split
from u2ucf import lmap, U2UCF, split_ratings, get_preferences, mapk, get_random_predictions, get_by_indexes

rating_median = 5

users_list = df_users.values.tolist()
ratings_list = df_ratings.values.tolist()


user_ratings = lmap(
    lambda x: [],
    range(len(df_users))
)


In [8]:
print(user_ratings)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],

In [7]:
transformed_count = 0
temp = 0
for rating in ratings_list:
    user_index = int(df_users.values[temp])
    item_index = int(df_books.values[temp])
    if user_index is not None and item_index is not None:
        transformed_count +=1
        user_ratings[user_index].append([item_index, rating[0] - rating_median])
        temp += 1

print(f"transformed {transformed_count} out of {len(ratings_list)} ratings")

user_ratings_train, user_ratings_test = split_ratings(user_ratings)

transformed 1031139 out of 1031139 ratings


For example, how ratings look now - they are stored very efficiently

In [8]:
user_ratings_train[:10]

[[],
 [],
 [[129002, -10.0]],
 [],
 [],
 [],
 [],
 [],
 [[129030, -10.0],
  [129042, -10.0],
  [129074, -5.0],
  [129007, -5.0],
  [129009, -10.0],
  [129051, -10.0],
  [129099, -3.0],
  [129089, -4.0],
  [129096, -4.0],
  [129036, -10.0],
  [129060, -5.0]],
 [[129122, -10.0], [129108, -10.0]]]

Creating model - very simple - only ratings in our special format needed 

In [9]:
recommender = U2UCF(user_ratings_train)

Some presentation logic to display results

In [10]:
from IPython.display import HTML, display

def get_full_ratings(user_ratings):
       
    return lmap(
        lambda x: {
            'book_index': x[0],
            'book_name': books_list[x[0]][1],
            'book_author': books_list[x[0]][2],
            'book_year': books_list[x[0]][3],
            'book_publisher': books_list[x[0]][4],
            'book_picture': '<img src="' + books_list[x[0]][5] + '" />',
            'rating': x[1] + rating_median
        },
        user_ratings
    )
      
    
def get_full_recommendations(user_index):
    recs = recommender.recommend(user_index)
    return lmap(
        lambda x: {
            'book_index': x,
            'book_name': books_list[x][1],
            'book_author': books_list[x][2],
            'book_year': books_list[x][3],
            'book_publisher': books_list[x][4],
            'book_picture': '<img src="' + books_list[x][5] + '" />'    
        },
        recs
    )

def get_full_recommendations_id_only(user_index):
    recs = recommender.recommend(user_index)
    return lmap(

        lambda x: x,
        recs
    )


def show_array_of_dicts(aod):
    if len(aod) == 0:
        return ""
    keys = aod[0].keys()

    header = "<tr>"
    for k in keys:
        header += "<td><b>" + str(k) + "</b></td>"
    header += "</tr>"

    body = ""

    for dct in aod:
        row = "<tr>"
        for k in keys:
           row += "<td>" + str(dct.get(k, "")) + "</td>"
        row += "</tr>"
        body += row
    
    html = "<table>" + header + body + "</table>"
    display(HTML(html))

And how recommendations look like (added train and test ratings for comparison):

In [11]:
user_idx = 1261

#train_ratings = get_full_ratings(user_ratings_train[user_idx])
#test_ratings = get_full_ratings(user_ratings_test[user_idx])
recommendations = get_full_recommendations_id_only(user_idx)

#display(HTML("<h3>Train ratings</h3><br />"))
#show_array_of_dicts(train_ratings)
#display(HTML("<h3>Test ratings</h3><br />"))
#show_array_of_dicts(test_ratings)
display(HTML("<h3>Recommendations</h3><br />"))
print(recommendations)


[175541, 183262, 151362, 165354, 138792, 131176, 131259, 218627, 208189, 172155, 144569, 136333, 183375, 20205, 188619, 142991, 215373, 93265, 212784, 149320]


In [15]:
import time
from tqdm import tqdm

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=BOSS;'
                      'Database=BooksModel;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

query = "SELECT DISTINCT UserId FROM Ratings;"
users_unique = pd.read_sql(query, conn)
users_unique = users_unique.values.tolist()

total_users = len(users_unique)
start_time = time.time()

for i, user in tqdm(enumerate(users_unique), total=total_users, miniters=1):

    recommendations = get_full_recommendations_id_only(user[0])
    cursor.execute("DELETE FROM UserRecommendations WHERE UserId=?", user[0])

    for recs in recommendations:
        cursor.execute("INSERT INTO UserRecommendations (UserId,ItemId) values(?,?)", user[0], recs)

    if i % (total_users // 10) == 0:
        progress = int((i / total_users) * 100)
        print("Progress: {}%".format(progress))


print("Progress: 100%")

end_time = time.time()
total_time = end_time - start_time
print("Total execution time: {:.2f} seconds".format(total_time))


conn.commit()
cursor. Close()

c:\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
  0%|          | 0/92106 [00:00<?, ?it/s]

In [98]:
import time
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

def process_user(user):
    recommendations = get_full_recommendations_id_only(user[0])
    cursor.execute("DELETE FROM UserRecommendations WHERE UserId=?", user[0])
    for recs in recommendations:
        cursor.execute("INSERT INTO UserRecommendations (UserId,ItemId) values(?,?)", user[0], recs)

def track_progress(num_done):
    progress = int((num_done / total_users) * 100)
    print("Progress: {}%".format(progress))

if __name__ == "__main__":
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=BOSS;'
                          'Database=BooksModel;'
                          'Trusted_Connection=yes;')
    cursor = conn.cursor()

    query = "SELECT DISTINCT UserId FROM Ratings;"
    users_unique = pd.read_sql(query, conn)
    users_unique = users_unique.values.tolist()

    total_users = len(users_unique)
    num_done = 0

    with Pool(cpu_count()) as p:
        for _ in tqdm(p.imap_unordered(process_user, users_unique), total=total_users, miniters=1):
            num_done += 1
            track_progress(num_done)

    print("Progress: 100%")
    end_time = time.time()
    total_time = end_time - start_time
    print("Total execution time: {:.2f} seconds".format(total_time))

    conn.commit()
    cursor.close()


c:\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
 31%|███       | 28404/92106 [00:00<00:00, 144361.16it/s]

Progress: 0%
Progress: 9%
Progress: 19%
Progress: 29%
Progress: 39%
Progress: 49%


100%|██████████| 92106/92106 [00:01<00:00, 79884.34it/s] 

Progress: 59%
Progress: 69%
Progress: 79%
Progress: 89%
Progress: 99%


Progress: 100%


Now let's get recommendations for 10000 random users and calculate predictivity metric of our algorithm:

Metric we plan to use is called $MAP@k$.

$$
MAP@k = \frac{1}{N} \sum_{u = 1}^N AP_u@k
$$
$$
AP_u@k = \frac{1}{\min(k, n_u)} \sum_{i=1}^k r_u(i) p_u@i
$$
$$p_u@k = \dfrac{1}{k}\sum_{j=1}^k r_u(j)$$


*   $N$ - Number of users.
*   $n_u$ - number of relevant items for user $u$ in test data.
*   $r_u(i)$ - binary value: 1 if item on postion $i$ belongs to relevant, otherwise 0.

In [15]:
rstate = 13

random_users, _ = train_test_split(list(range(len(df_users))), train_size=10000, random_state=rstate, shuffle=True)
test_preferences = get_preferences(get_by_indexes(user_ratings_test, random_users))

recommendations = lmap(recommender.recommend, random_users)

mapk_metric = mapk(test_preferences, recommendations)

print(f"mapk for our model: {mapk_metric}")



mapk for our model: 0.0


For comparison calculate the same metric for random algorithm, which just recommends random non-rated items and see ratio - predictive power of our algorithm in comparison to random one.

In [16]:
train_preferences = get_preferences(get_by_indexes(user_ratings_train, random_users))

random_predictions = get_random_predictions(train_preferences, len(df_books), 20, rstate)

random_mapk_metric = mapk(test_preferences, random_predictions)

print(f"mapk for random predictions: {random_mapk_metric}")

if random_mapk_metric != 0:
    print(f"our model/random predictions ratio: {mapk_metric/random_mapk_metric}")

mapk for random predictions: 0.0
